---
# <center> <font size="15"> <font color='red'>**SSH connect to get only useful SOLEDGE data on Marconi**</font></font>
---

---
## <center> <font color='xkcd:cherry red'>**Authentification**</font>
---

In [9]:
import paramiko
import h5py
from stat import S_ISDIR, S_ISREG
import numpy as np

# Get password
#password = getpass.getpass(prompt='Password: ', stream=None)
password = 'Theinetrobot4428!!'

# Create SSH client
ssh = paramiko.SSHClient()

## Add missing host key
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

## Connect to the server
def connect():
    try:
        ssh.connect(hostname='login.marconi.cineca.it', username='rvarenne', password=password)
    except:
        print('Connection failed. Please try again.')
        return False
    return True

## Print all file in remote directory
def print_all_files_in_remote_directory(directory):
    sftp = ssh.open_sftp()
    for entry in sftp.listdir_attr(directory):
        mode = entry.st_mode
        if S_ISDIR(mode):
            print(entry.filename + " is folder")
        elif S_ISREG(mode):
            print(entry.filename + " is file")

## Print all keys of hdf5 file in remote directory
def print_keys_of_hdf5_file_in_remote_directory(directory, filename):
    sftp = ssh.open_sftp()
    f = sftp.open(directory+'/'+filename, 'r')
    print(f)
    fh5 = h5py.File(f, 'r')
    print(fh5.keys())

## Print a numpy array of a specific dataset in hdf5 file in remote directory
def print_numpy_array_of_dataset_in_hdf5_file_in_remote_directory(directory, filename, dataset):
    sftp = ssh.open_sftp()
    f = sftp.open(directory+'/'+filename, 'r')
    fh5 = h5py.File(f, 'r')
    print( np.array( fh5.get(dataset) ) )

test_directory='/marconi_scratch/userexternal/rvarenne/SOLEDGE_TEST_CASE1/circular_noEIRENE_3D_gloria_version/run_dir'
test_filename='plasmaFinal.h5'
#test_filename='mesh_raptorX.h5'
test_dataset='zone1/ExB/psi'

if connect():
    print_all_files_in_remote_directory(test_directory)
    print_keys_of_hdf5_file_in_remote_directory(test_directory, test_filename)
    print_numpy_array_of_dataset_in_hdf5_file_in_remote_directory(test_directory, test_filename, test_dataset)
    #ssh.close()

mesh_raptorX.h5 is file
soledge3x is file
dVdS_raptorX.h5 is file
split_raptorX.h5 is file
metric_raptorX.h5 is file
timeInfo.txt is file
refParam_raptorX.h5 is file
balances_0 is file
sources_raptorX.h5 is file
masks_raptorX.h5 is file
param_raptorX.txt is file
species_raptorX.txt is file
balances_1 is file
param_geom.txt is file
plasmaFinal.h5 is file
Plasma is folder
soledge3x.status is file
<KeysViewHDF5 ['NZones', 'Nelts', 'Nspecies', 'core', 'dt', 'metadata', 'tExec', 'tExecCumu', 'tNextSave', 'tcpu', 'tcpuCumu', 'tsimu', 'zone1', 'zone2']>
[[[ 1.23511257e-05  1.24455159e-05  1.24972011e-05 ...  1.24455159e-05
    1.24972011e-05  1.23062044e-05]
  [ 5.35618430e-06  5.36020489e-06  5.46588756e-06 ...  5.36020489e-06
    5.46588756e-06  5.43286753e-06]
  [-3.98699492e-06 -4.08891471e-06 -4.03048851e-06 ... -4.08891471e-06
   -4.03048851e-06 -3.94567233e-06]
  ...
  [ 5.35618430e-06  5.36020489e-06  5.46588756e-06 ...  5.36020489e-06
    5.46588756e-06  5.43286753e-06]
  [-3.9869949

---
## <center> <font color='xkcd:cherry red'>**Proof of concept : get $\langle \mathbf{E}\times\mathbf{B} \cdot \nabla \theta \rangle_{FS}$ and plot radial profile**</font>
---

In [43]:
## Get the flux averaged value of a specific dataset in hdf5 file in remote directory
def get_FS_profile(directory, filename, dataset):
    sftp = ssh.open_sftp()
    f = sftp.open(directory+'/'+filename, 'r')
    fh5 = h5py.File(f, 'r')
    data = np.mean( np.array( fh5.get(dataset) ) , axis=(1,2))
    fh5.close()
    return data
    
    
#directory='/marconi_scratch/userexternal/rvarenne/SOLEDGE_TEST_CASE1/circular_noEIRENE_3D_gloria_version/run_dir'
directory='/marconi_scratch/userexternal/gfalchet/S3XE/SE3Xneo/3Dneo_from52thVQ/neo3D_r13/run_dir'
filename='plasmaFinal.h5'
dataset='zone1/ExB/theta'

data = None
if connect():
    data = get_FS_profile(directory, filename, dataset)
    ssh.close()

print(data)

import matplotlib.pyplot as plt
plt.plot(data)
plt.axhline(y=0, color='k', linestyle='--')


[ 0.01052762  0.01052762  0.01052762  0.0129445   0.01614837  0.01627673
  0.01494804  0.01348836  0.0121756   0.0114771   0.01102322  0.00849734
  0.00576002  0.00597137  0.00746237  0.00901014  0.00991698  0.01017122
  0.01140358  0.01408398  0.01648685  0.0165089   0.01518154  0.01495031
  0.01594043  0.01713661  0.01703286  0.01429193  0.00977708  0.00685969
  0.00770162  0.00962247  0.00837601  0.0020465  -0.00485154 -0.00653058
 -0.00397146  0.0005301   0.00539737  0.01031795  0.01585945  0.01972021
  0.02073001  0.02124041  0.02176272  0.02021165  0.01650272  0.01307254
  0.01092008  0.01131634  0.01661279  0.02633982  0.03600856  0.04039979
  0.04025953  0.03768456  0.03038063  0.01681797  0.00154598 -0.00554985
 -0.0036766  -0.00026358 -0.00534034 -0.01402772]


---
## <center> <font color='xkcd:cherry red'>**Construct the time averaged $\langle \mathbf{E}\times\mathbf{B} \cdot \nabla \theta \rangle_{FS}$ and plot radial profile**</font>
---

In [10]:
from joblib import Parallel, delayed
import dill as pickle

## Open a SFTP connection
sftp = ssh.open_sftp()

directory='/marconi_scratch/userexternal/gfalchet/S3XE/SE3Xneo/3Dneo_from52thVQ/neo3D_r13/run_dir/Plasma'

## Get the list of filepath of Plasma_xxx.h5
filename_ar=sorted(sftp.listdir(directory))
print(filename_ar)

## Create a filepath array by adding the directory path to the filename for each element of filename_ar
filepath_ar = [directory+'/'+filename for filename in filename_ar]

i = 1
## Define a function that takes a file path (to the Plasma_xxx.h5) as an input and return the flux surface averaged value of ExB.e_theta
def get_FS_profile(filepath):
    global i
    print('Iteration %s / %s' % (i, len(filepath_ar)))
    i += 1
    # print('In the function !')
    f = sftp.open(filepath, 'r')
    # print('Passed the sftp thingy !')
    fh5 = h5py.File(f, 'r')
    # print('fh5 is opened !')
    data = np.mean( np.array( fh5.get('zone1/ExB/theta') ) , axis=(1,2))
    # print('Big calculation done !')
    fh5.close()
    # print('No it is not crashed !')
    return data


## Map the get_FS_profile function to the filepath_ar array
if connect():
    Er_time_trace = list(map(get_FS_profile, filepath_ar[:25]))

    #Er_time_trace = Parallel(n_jobs=6, prefer="threads")(delayed(get_FS_profile)(x) for x in filepath_ar[:5])

np.save('Er_time_trace.npy', Er_time_trace)

['plasma_00000.h5', 'plasma_00001.h5', 'plasma_00002.h5', 'plasma_00003.h5', 'plasma_00004.h5', 'plasma_00005.h5', 'plasma_00006.h5', 'plasma_00007.h5', 'plasma_00008.h5', 'plasma_00009.h5', 'plasma_00010.h5', 'plasma_00011.h5', 'plasma_00012.h5', 'plasma_00013.h5', 'plasma_00014.h5', 'plasma_00015.h5', 'plasma_00016.h5', 'plasma_00017.h5', 'plasma_00018.h5', 'plasma_00019.h5', 'plasma_00020.h5', 'plasma_00021.h5', 'plasma_00022.h5', 'plasma_00023.h5', 'plasma_00024.h5', 'plasma_00025.h5', 'plasma_00026.h5', 'plasma_00027.h5', 'plasma_00028.h5', 'plasma_00029.h5', 'plasma_00030.h5', 'plasma_00031.h5', 'plasma_00032.h5', 'plasma_00033.h5', 'plasma_00034.h5', 'plasma_00035.h5', 'plasma_00036.h5', 'plasma_00037.h5', 'plasma_00038.h5', 'plasma_00039.h5', 'plasma_00040.h5', 'plasma_00041.h5', 'plasma_00042.h5', 'plasma_00043.h5', 'plasma_00044.h5', 'plasma_00045.h5', 'plasma_00046.h5', 'plasma_00047.h5', 'plasma_00048.h5', 'plasma_00049.h5', 'plasma_00050.h5', 'plasma_00051.h5', 'plasma_000

In [77]:
print(Er_time_trace[2])

IndexError: list index out of range

In [78]:
## Mini test of parallelisation
from multiprocess import Pool
import numpy as np

def f(x):
    import numpy as np
    return np.array([x, x**2])

p = Pool(5)
t=p.map(f, np.arange(10))
print(t)


[array([0, 0]), array([1, 1]), array([2, 4]), array([3, 9]), array([ 4, 16]), array([ 5, 25]), array([ 6, 36]), array([ 7, 49]), array([ 8, 64]), array([ 9, 81])]


In [2]:
from joblib import Parallel, delayed

def f(x):
    return x + 2

numbers1 = Parallel(n_jobs=2, prefer="threads")(delayed(f)(x) for x in range(10))

print(numbers1)

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
